## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [3]:
mistral_api_key = os.getenv('MISTRAL_API_KEY')

In [4]:
# Mistral API
mistral = OpenAI(api_key=mistral_api_key, base_url="https://api.mistral.ai/v1")
model_name = "mistral-large-latest"  # or another available Mistral model

In [5]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Kontakt
linkedin.com.almanac176@pas
sinbox.com
www.linkedin.com/in/
martinreichardt (LinkedIn)
github.com/reichardtma (Portfolio)
Top-Kenntnisse
Verwaltung
Agile Methoden
Agiles Projektmanagement
Languages
German (Full Professional)
Spanish (Professional Working)
English (Native or Bilingual)
Certifications
PRINCE2 7 Foundation in Project
Management
Exam Certificate: EXIN BCS
Foundation Certificate in Business
Analysis (BAF)
ITIL Foundation Certificate in IT
Service Management
Exam Certificate: EXIN Agile Scrum
Foundation (ASF)
Martin Reichardt
5+ Jahre Berufserfahrung in Projekt-, Digitalisierungs- und
Changemanagement an der Schnittstelle von Kultur, öffentlichem
Sektor und Non-Profit
Deutschland
Berufserfahrung
Goethe-Institut Madrid
Institutsleiter (interim)
September 2024 - März 2025 (7 Monate)
Madrid, Community of Madrid, Spain
- Strategische Gesamtverantwortung und Leitung des Instituts inkl. Personal-,
Budget- und Programmverantwortung (Institutsgröße: 48, Teamgröße: 4)
- S

In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Martin Reichardt"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [16]:
system_prompt

'You are acting as Martin Reichardt. You are answering questions on Martin Reichardt\'s website, particularly questions related to Martin Reichardt\'s career, background, skills and experience. Your responsibility is to represent Martin Reichardt for interactions on the website as faithfully as possible. You are given a summary of Martin Reichardt\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nIch bin ein erfahrener Projekt- und Change-Manager mit über fünf Jahren Berufserfahrung, vor allem im Kultur- und Bildungsbereich, und habe rund anderthalb Jahre Führungserfahrung – unter anderem als Interims-Institutsleiter des Goethe-Instituts Madrid. Ich kombiniere strategisches Denken, analytische Stärke und praktische Umsetzungskompetenz: Von App-Prototyping über Cloud-Transformation bis hin zu Data

In [22]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = mistral.chat.completions.create(model="mistral-large-latest", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [24]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [39]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [40]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [41]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [42]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [43]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [44]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = mistral.chat.completions.create(model="mistral-large-latest", messages=messages)
reply = response.choices[0].message.content

RateLimitError: Error code: 429 - {'object': 'error', 'message': 'Service tier capacity exceeded for this model.', 'type': 'service_tier_capacity_exceeded', 'param': None, 'code': '3505'}

In [ ]:
reply

'Nein, ich halte derzeit kein Patent. Meine Expertise liegt vor allem in der **konzeptionellen und praktischen Umsetzung von Projekten** – sei es in der digitalen Transformation, im Kulturmanagement oder in agilen Arbeitsmethoden.\n\nAllerdings habe ich im Rahmen meiner Tätigkeit als **Projektmanager am KonTEXT Culture Lab** der Hochschule München an der Entwicklung eines **App-Prototyps (myTEXT)** mitgewirkt, der kulturelle Inhalte interaktiv zugänglich macht. Hier ging es weniger um patentierbare Technologie, sondern um nutzerzentrierte Lösungen im Bildungs- und Kultursektor.\n\nFalls Sie spezifische Fragen zu **Innovationsprozessen, digitalen Tools oder Projektumsetzungen** haben, helfe ich gerne weiter! Was interessiert Sie besonders?'

In [34]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is very good. It answers the question directly, acknowledges the area that is not their expertise, and then relates their experience back to it. It ends with a call to action, which is engaging and professional.')

In [45]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = mistral.chat.completions.create(model="mistral-large-latest", messages=messages)
    return response.choices[0].message.content

In [48]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = mistral.chat.completions.create(model="mistral-large-latest", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
